In [1]:
import pandas as pd

In [2]:
import pandas as pd
import numpy as np

#NLTK
import nltk
from nltk import word_tokenize, WordPunctTokenizer, regexp_tokenize
from nltk import word_tokenize, WordPunctTokenizer, regexp_tokenize

#Plotting 
import matplotlib.pyplot as plt


#Keras
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


2023-05-04 01:17:35.957787: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-04 01:17:36.006044: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-04 01:17:36.006729: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 01:17:36.824912: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
train_yes_es = pd.read_json('training/EXIST2023_training.json')
train_yes_es=train_yes_es.transpose()
train_yes_es=train_yes_es.loc[train_yes_es['lang'].apply(lambda x:x=='es')]
train_yes_es=train_yes_es.loc[train_yes_es['labels_task1'].apply(lambda x:x.count('YES')>3)]
train_yes_es['label1']='YES'

#NO

train_no_es = pd.read_json('training/EXIST2023_training.json')
train_no_es=train_no_es.transpose()
train_no_es=train_no_es.loc[train_no_es['lang'].apply(lambda x:x=='es')]
train_no_es=train_no_es.loc[train_no_es['labels_task1'].apply(lambda x:x.count('NO')>3)]
train_no_es['label1']='NO'

In [18]:
validation_yes_es = pd.read_json('dev/EXIST2023_dev.json')
validation_yes_es=validation_yes_es.transpose()
validation_yes_es=validation_yes_es.loc[validation_yes_es['lang'].apply(lambda x:x=='es')]
validation_yes_es=validation_yes_es.loc[validation_yes_es['labels_task1'].apply(lambda x:x.count('YES')>3)]
validation_yes_es['label1']='YES'

#NO

validation_no_es = pd.read_json('dev/EXIST2023_dev.json')
validation_no_es=validation_no_es.transpose()
validation_no_es=validation_no_es.loc[validation_no_es['lang'].apply(lambda x:x=='es')]
validation_no_es=validation_no_es.loc[validation_no_es['labels_task1'].apply(lambda x:x.count('NO')>3)]
validation_no_es['label1']='NO'

In [20]:
frames = [validation_yes_es, validation_no_es]
validation = pd.concat(frames)
validation.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,300002,es,@anacaotica88 @MordorLivin No me acuerdo de lo...,6,"[Annotator_731, Annotator_732, Annotator_315, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, YES]","[JUDGEMENTAL, REPORTED, -, JUDGEMENTAL, JUDGEM...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_ES,YES
1,300004,es,Also mientras les decia eso la señalaba y deci...,6,"[Annotator_259, Annotator_739, Annotator_291, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, JUDGEMENTAL,...","[[-], [SEXUAL-VIOLENCE], [SEXUAL-VIOLENCE], [S...",DEV_ES,YES
2,300007,es,@DavidGR18 @pppbernat @abc_es @agarzon @IreneM...,6,"[Annotator_731, Annotator_732, Annotator_315, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, YES]","[DIRECT, DIRECT, -, DIRECT, JUDGEMENTAL, REPOR...","[[IDEOLOGICAL-INEQUALITY, SEXUAL-VIOLENCE, MIS...",DEV_ES,YES
3,300008,es,@DavidArranzVox @AnabelAlonso_of Uyyy a q huel...,6,"[Annotator_742, Annotator_743, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, YES]","[JUDGEMENTAL, JUDGEMENTAL, -, REPORTED, JUDGEM...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_ES,YES
4,300010,es,@kokreto84 @Play87834898 @venusoncrack Me gust...,6,"[Annotator_744, Annotator_745, Annotator_746, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, REPORTED, DIRECT, JUDGEMENTAL, DIRECT...","[[MISOGYNY-NON-SEXUAL-VIOLENCE], [STEREOTYPING...",DEV_ES,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
485,300534,es,El Gobernador @samuel_garcias se da el lujo d...,6,"[Annotator_726, Annotator_727, Annotator_357, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, NO, NO, NO]","[-, REPORTED, JUDGEMENTAL, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY], [STEREOTYPING-...",DEV_ES,NO
486,300538,es,Violencia simbólica también es un Presidente q...,6,"[Annotator_750, Annotator_751, Annotator_189, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",DEV_ES,NO
487,300539,es,En el #podcast Un día de libros 67: Violencia ...,6,"[Annotator_754, Annotator_320, Annotator_123, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, REPORTED, -, -, -, -]","[[-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-], [-]...",DEV_ES,NO
488,300540,es,Top story: NBA tiene menos actos de violencia ...,6,"[Annotator_259, Annotator_739, Annotator_291, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, JUDGEMENTAL, -]","[[-], [-], [-], [-], [MISOGYNY-NON-SEXUAL-VIOL...",DEV_ES,NO


In [4]:
frames = [train_yes_es, train_no_es]
train = pd.concat(frames)
train.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...",6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, NO, YES, YES, YES]","[REPORTED, JUDGEMENTAL, -, REPORTED, JUDGEMENT...","[[OBJECTIFICATION], [OBJECTIFICATION, SEXUAL-V...",TRAIN_ES,YES
1,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,6,"[Annotator_19, Annotator_20, Annotator_21, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, YES, NO, YES, YES]","[REPORTED, -, JUDGEMENTAL, -, JUDGEMENTAL, DIR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [-...",TRAIN_ES,YES
2,100008,es,@BestKabest Esta gringa sigue llorando por el ...,6,"[Annotator_25, Annotator_26, Annotator_27, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, JUDGEMENTAL, DIRECT, ...","[[IDEOLOGICAL-INEQUALITY], [STEREOTYPING-DOMIN...",TRAIN_ES,YES
3,100028,es,"@ShahidForChange @TeamPelosi Quiet, sexist ^%$...",6,"[Annotator_109, Annotator_110, Annotator_111, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, NO, YES, YES, YES]","[JUDGEMENTAL, -, -, DIRECT, DIRECT, DIRECT]","[[STEREOTYPING-DOMINANCE], [-], [-], [IDEOLOGI...",TRAIN_ES,YES
4,100036,es,@Harassed_girl loca d mierda en k momento,6,"[Annotator_115, Annotator_116, Annotator_117, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, NO]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, -]","[[STEREOTYPING-DOMINANCE, SEXUAL-VIOLENCE], [M...",TRAIN_ES,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
3189,103616,es,"@petrogustavo Doctor ... repito, los niños que...",6,"[Annotator_139, Annotator_140, Annotator_141, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, NO, YES, NO, YES, NO]","[-, -, REPORTED, -, DIRECT, -]","[[-], [-], [SEXUAL-VIOLENCE], [-], [IDEOLOGICA...",TRAIN_ES,NO
3190,103619,es,"La Argentina explotada por todos lados, pero p...",6,"[Annotator_313, Annotator_314, Annotator_315, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_ES,NO
3191,103620,es,Hoy con testigos el Presidente López violó la ...,6,"[Annotator_229, Annotator_230, Annotator_231, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_ES,NO
3192,103639,es,cahcake ya casi queda poco para el años zorra ...,6,"[Annotator_211, Annotator_212, Annotator_213, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, YES, YES, NO, NO, NO]","[-, DIRECT, JUDGEMENTAL, -, -, -]","[[-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [OBJECTI...",TRAIN_ES,NO


In [5]:
import re
hash_regex = re.compile(r"#(\w+)")
hstgs = [] # To store the hashtags so we can exclude them from some parts of the analysis
def hash_repl(match):
    _ = '__HASH_'+match.group(1).upper()
    hstgs.append(_)
    return _

In [6]:
user_regex = re.compile(r"@(\w+)")
usr_names = [] # To store the user names so we can exclude them from some parts of the analysis
def user_repl(match):
    _ = '__user_'+match.group(1).upper()
    usr_names.append(_)
    return _

In [7]:
url_regex = re.compile(r"(http|https|ftp)://[a-zA-Z0-9\./]+")
def url_repl(match):
    return '__URL_'

In [8]:
# Repeating words like hurrrryyyyyy
rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE);
def rpt_repl(match):
    return match.group(1)+match.group(1)


In [9]:
# Spliting by word boundaries
word_bound_regex = re.compile(r"\W+")

# Punctuations
punctuations = \
	[	#('',		['.', ] )	,\
		#('',		[',', ] )	,\
		#('',		['\'', '\"', ] )	,\
		('__PUNC_EXCL',		['!', '¡', ] )	,\
		('__PUNC_QUES',		['?', '¿', ] )	,\
		('__PUNC_ELLP',		['...', '…', ] )	,\
	]

#For punctuation replacement
def punctuations_repl(match):
	text = match.group(0)
	repl = []
	for (key, parr) in punctuations :
		for punc in parr :
			if punc in text:
				repl.append(key)
	if( len(repl)>0 ) :
		return ' '+' '.join(repl)+' '
	else :
		return ' '

In [10]:

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [11]:
def sb_stem(text, only_first=0):
    text = [word if(word[0:2]=='__') else word.lower() for word in text.split() if ((len(word) >= 3) or (word in ['no','si', 'sí', 'ni']))] #   If we are doing negation analysis, maybe is a better idea to keep the small words (like 'no')
    text = [stemmer.stem(w) if w[0:2]!='__' else w for w in text ]
    
    return(text)

In [12]:
def processAll(text):
    text = re.sub( hash_regex, hash_repl, text )
    #text = re.sub( user_regex, user_repl, text)
    text = re.sub( url_regex, url_repl, text )
    
    text = text.replace('\'','')
    
    text = re.sub( word_bound_regex , punctuations_repl, text )
    text = re.sub( rpt_regex, rpt_repl, text )
    
    text = sb_stem(text)    
    return text

In [21]:
train['processed_tweet'] = train.tweet.apply(processAll)
validation['processed_tweet'] = validation.tweet.apply(processAll)

In [14]:
train.head()

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1,processed_tweet
100001,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...",6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, NO, YES, YES, YES]","[REPORTED, JUDGEMENTAL, -, REPORTED, JUDGEMENT...","[[OBJECTIFICATION], [OBJECTIFICATION, SEXUAL-V...",TRAIN_ES,YES,"[thechiflis, ignor, otro, capull, problem, con..."
100005,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,6,"[Annotator_19, Annotator_20, Annotator_21, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, YES, NO, YES, YES]","[REPORTED, -, JUDGEMENTAL, -, JUDGEMENTAL, DIR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [-...",TRAIN_ES,YES,"[novadragon21, icep4ck, tvdannyz, entonc, com,..."
100008,100008,es,@BestKabest Esta gringa sigue llorando por el ...,6,"[Annotator_25, Annotator_26, Annotator_27, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, JUDGEMENTAL, DIRECT, ...","[[IDEOLOGICAL-INEQUALITY], [STEREOTYPING-DOMIN...",TRAIN_ES,YES,"[bestkabest, esta, gring, sig, llor, por, game..."
100028,100028,es,"@ShahidForChange @TeamPelosi Quiet, sexist ^%$...",6,"[Annotator_109, Annotator_110, Annotator_111, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, NO, YES, YES, YES]","[JUDGEMENTAL, -, -, DIRECT, DIRECT, DIRECT]","[[STEREOTYPING-DOMINANCE], [-], [-], [IDEOLOGI...",TRAIN_ES,YES,"[shahidforchang, teampelosi, quiet, sexist, __..."
100036,100036,es,@Harassed_girl loca d mierda en k momento,6,"[Annotator_115, Annotator_116, Annotator_117, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, NO]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, -]","[[STEREOTYPING-DOMINANCE, SEXUAL-VIOLENCE], [M...",TRAIN_ES,YES,"[harassed_girl, loc, mierd, moment]"


In [24]:
train_data = pd.DataFrame()
train_data['tweet'] = train['processed_tweet']
train_data['label'] = train['label1']
train_data.reset_index(inplace=True, drop = True)
train_data

,tweet,label
0,"[thechiflis, ignor, otro, capull, problem, con...",YES
1,"[novadragon21, icep4ck, tvdannyz, entonc, com,...",YES
2,"[bestkabest, esta, gring, sig, llor, por, game...",YES
3,"[shahidforchang, teampelosi, quiet, sexist, __...",YES
4,"[harassed_girl, loc, mierd, moment]",YES
...,...,...
3189,"[petrogustav, doctor, __PUNC_ELLP, repit, los,...",NO
3190,"[argentin, explot, por, tod, lad, per, pag, de...",NO
3191,"[hoy, con, testig, president, lopez, viol, ley...",NO
3192,"[cahcak, casi, qued, poc, par, años, zorr, sol...",NO


In [25]:
validation_data = pd.DataFrame()
validation_data['tweet'] = validation['processed_tweet']
validation_data['label'] = validation['label1']
validation_data.reset_index(inplace=True, drop = True)
validation_data

,tweet,label
0,"[anacaotica88, mordorlivin, acuerd, los, detal...",YES
1,"[also, mientr, les, deci, eso, señal, deci, qu...",YES
2,"[davidgr18, ppbernat, abc_es, agarzon, irenemo...",YES
3,"[davidarranzvox, anabelalonso_of, uyy, huel, _...",YES
4,"[kokreto84, play87834898, venusoncrack, gust, ...",YES
...,...,...
485,"[gobern, samuel_garci, luj, sub, vide, complet...",NO
486,"[violenci, simbol, tambien, president, que, to...",NO
487,"[__HASH_PODCAST, dia, libr, violenci, simbol, ...",NO
488,"[top, story, nba, tien, men, actos, violenci, ...",NO


In [27]:
print(max(train_data['tweet'].apply(len)))
print(max(validation_data['tweet'].apply(len)))

50
48


In [15]:
from gensim.models import KeyedVectors


In [16]:
vecs = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.txt', binary=False)

In [33]:
def vectorize_tweet(tweet):
    vec = []
    for word in tweet:
        if word in vecs.key_to_index:
            vec.append(vecs[word])
    return vec

In [34]:
train_data['tweet'] = train_data['tweet'].apply(vectorize_tweet)
validation_data['tweet'] = validation_data['tweet'].apply(vectorize_tweet)

42
43


In [38]:
import random

X_train = list(train_data['tweet'])
Y_train = list(train_data['label'])

X_test = list(validation_data['tweet'])
Y_test = list(validation_data['label'])

temp = list(zip(X_train, Y_train))
random.shuffle(temp)
X_train, Y_train = zip(*temp)
X_train = list(X_train)
Y_train = list(Y_train)

In [41]:
import tensorflow as tf

In [42]:
def padto43(x):
    if len(x) < 43:
        for i in range(43 - len(x)):
            x.append([0]*300)
    return x

In [44]:
for i in range(len(X_train)):
    X_train[i] = padto43(X_train[i])

for i in range(len(X_test)):
    X_test[i] = padto43(X_test[i])

In [45]:
X_train_tensor = tf.convert_to_tensor(X_train)
X_test_tensor = tf.convert_to_tensor(X_test)

2023-05-04 01:29:31.540437: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-04 01:29:31.541803: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [46]:
print(tf.shape(X_train_tensor))
print(tf.shape(X_test_tensor))

tf.Tensor([3194   43  300], shape=(3,), dtype=int32)
tf.Tensor([490  43 300], shape=(3,), dtype=int32)


In [47]:
#convert yes to 1 and no to 0
for i in range(len(Y_train)):
    if Y_train[i] == 'YES':
        Y_train[i] = [1, 0]
    else:
        Y_train[i] = [0, 1]

for i in range(len(Y_test)):
    if Y_test[i] == 'YES':
        Y_test[i] = [1, 0]
    else:
        Y_test[i] = [0, 1]

In [48]:

Y_train_tensor = tf.convert_to_tensor(Y_train)
Y_test_tensor = tf.convert_to_tensor(Y_test)

In [52]:
MLmodel = tf.keras.Sequential([
    tf.keras.layers.LSTM(150),
    tf.keras.layers.Dense(2, activation='softmax')
])
MLmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
MLmodel.build(input_shape=(None, 43, 300))
MLmodel.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 150)               270600    
                                                                 
 dense_1 (Dense)             (None, 2)                 302       
                                                                 
Total params: 270,902
Trainable params: 270,902
Non-trainable params: 0
_________________________________________________________________


2023-05-04 01:31:00.775216: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:31:00.776890: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:31:00.778114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [54]:
num_epochs = 50
MLmodel.fit(X_train_tensor, Y_train_tensor, epochs=num_epochs, validation_data=(X_test_tensor, Y_test_tensor))

Epoch 1/50
100/100 [==============================] - 3s 29ms/step - loss: 0.5064 - accuracy: 0.7595 - val_loss: 0.6136 - val_accuracy: 0.7000
Epoch 2/50
100/100 [==============================] - 3s 30ms/step - loss: 0.5060 - accuracy: 0.7655 - val_loss: 0.5916 - val_accuracy: 0.6796
Epoch 3/50
100/100 [==============================] - 3s 28ms/step - loss: 0.4882 - accuracy: 0.7774 - val_loss: 0.6248 - val_accuracy: 0.6735
Epoch 4/50
100/100 [==============================] - 3s 28ms/step - loss: 0.4938 - accuracy: 0.7796 - val_loss: 0.5868 - val_accuracy: 0.7082
Epoch 5/50
100/100 [==============================] - 3s 28ms/step - loss: 0.4815 - accuracy: 0.7877 - val_loss: 0.6247 - val_accuracy: 0.6878
Epoch 6/50
100/100 [==============================] - 3s 28ms/step - loss: 0.4793 - accuracy: 0.7924 - val_loss: 0.6030 - val_accuracy: 0.6939
Epoch 7/50
100/100 [==============================] - 3s 28ms/step - loss: 0.4763 - accuracy: 0.7959 - val_loss: 0.5967 - val_accuracy: 0.6735

In [58]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, 43, 300))
model.summary()


2023-05-04 01:37:22.362058: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:37:22.363712: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:37:22.364947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 300)              541200    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 16)                4816      
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 2)                 34        
                                                                 
Total params: 546,050
Trainable params: 546,050
Non-trainable params: 0
________________________________________________

2023-05-04 01:37:22.564145: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:37:22.565659: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_1_grad/concat/split_1/split_dim' with dtype int32
	 [[{{node gradients/split_1_grad/concat/split_1/split_dim}}]]


In [59]:
 num_epochs = 50
model.fit(X_train_tensor, Y_train_tensor, epochs=num_epochs, validation_data=(X_test_tensor, Y_test_tensor))


Epoch 1/50


2023-05-04 01:37:40.556548: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:37:40.558336: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:37:40.559643: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

 99/100 [============================>.] - ETA: 0s - loss: 0.6736 - accuracy: 0.5814

2023-05-04 01:37:47.885984: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:37:47.887625: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:37:47.888837: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

100/100 [==============================] - 8s 48ms/step - loss: 0.6740 - accuracy: 0.5808 - val_loss: 0.6919 - val_accuracy: 0.5653
Epoch 2/50
100/100 [==============================] - 4s 42ms/step - loss: 0.6508 - accuracy: 0.6218 - val_loss: 0.6243 - val_accuracy: 0.6490
Epoch 3/50
100/100 [==============================] - 4s 45ms/step - loss: 0.6186 - accuracy: 0.6528 - val_loss: 0.6210 - val_accuracy: 0.6755
Epoch 4/50
100/100 [==============================] - 4s 44ms/step - loss: 0.6084 - accuracy: 0.6703 - val_loss: 0.6326 - val_accuracy: 0.6449
Epoch 5/50
100/100 [==============================] - 5s 48ms/step - loss: 0.5917 - accuracy: 0.6882 - val_loss: 0.6063 - val_accuracy: 0.6633
Epoch 6/50
100/100 [==============================] - 4s 45ms/step - loss: 0.5845 - accuracy: 0.6935 - val_loss: 0.6019 - val_accuracy: 0.6755
Epoch 7/50
100/100 [==============================] - 4s 41ms/step - loss: 0.5693 - accuracy: 0.7120 - val_loss: 0.6234 - val_accuracy: 0.6449
Epoch 8/50